# Chapter 2: メトロポリス・ヘイスティングス法 - 直感的理解から実装まで

## 学習目標
- メトロポリス・ヘイスティングス法の**革新的アイデア**を直感的に理解する
- 受理確率の導出と**数学的美しさ**を体感する
- 提案分布の選択と**探索効率**への影響を定量的に評価できる
- 実装と性能評価方法を習得し、**実践的診断スキル**を身につける
- 様々な分布からのサンプリング例を通じて**汎用性**を体験する

## 🚀 なぜMH法は革命的なのか？

### 従来の数値積分との決定的違い

**従来の数値積分**:
- 格子点を規則的に配置してf(x)を評価
- 次元が増えると格子点数が指数的に増加（次元の呪い）
- 10次元で各軸100点→10^20 格子点（現実的に不可能）

**MH法の革新**:
- **適応的サンプリング**: 重要な領域により多くの点を配置
- **確率的探索**: ランダムウォークによる効率的な空間探索
- **詳細釣り合い条件**: 目標分布に確実に収束する理論保証

### 「賢いランダムウォーク」としての直感

MH法は、**確率地形図**を見ながら歩く賢いハイカーです：

- 🏔️ **高い確率の領域**（山頂）: 積極的に向かう
- 🏜️ **低い確率の領域**（谷）: 時々訪れるが短時間
- 🎯 **バランス**: 各領域の滞在時間 ∝ その領域の確率

このバランスを実現するのが**受理確率α**の巧妙な設計です。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
np.random.seed(42)

## 2.1 メトロポリス・ヘイスティングス法：MCMCの最も基本的で汎用的なアルゴリズム

メトロポリス・ヘイスティングス法（MH法）は、MCMCの中でも最も基本的かつ汎用性の高いアルゴリズムです。詳細釣り合い条件という強力な設計指針を具体的なアルゴリズムとして実装した、真に革命的な手法です。

### MH法の核心的アイデア

MH法が画期的である理由は、その巧妙な設計にあります：

1. **正規化定数の相殺**: 目標分布の比率を使うことで、計算困難な正規化定数が相殺される
2. **詳細釣り合いの充足**: 採択/棄却メカニズムが詳細釣り合い条件を自動的に満たす
3. **汎用性**: 目標分布の密度関数（正規化不要）が計算できれば適用可能

### アルゴリズムの流れ

1. **初期化**: パラメータの初期値 $x^{(t)}$ を設定
2. **提案**: 提案分布 $q(x'|x^{(t)})$ から次の状態候補 $x'$ をサンプリング
3. **受理確率の計算**:
   $$\alpha = \min\left(1, \frac{p(x')q(x^{(t)}|x')}{p(x^{(t)})q(x'|x^{(t)})}\right)$$
4. **採択/棄却**: 確率 $\alpha$ で提案を採択、そうでなければ現在の状態に留まる
5. **繰り返し**: ステップ2に戻る

### 「賢いランダムウォーク」としての直感

MH法は、目標分布という「地形図」を常に見ながら歩く、賢いランダムウォークと見なせます：
- **確率の高い場所**（標高の高い場所）へは積極的に移動（高い採択確率）
- **確率の低い場所**（標高の低い場所）へはためらいながら移動（低い採択確率）

この「賢さ」を実装しているのが採択確率 $\alpha$ です。

## 2.2 MH法の最も巧妙な点：正規化定数の相殺

### ベイズ推論における問題の再確認

ベイズ推論で困っていたのは、事後分布の正規化定数が計算できないことでした：

$$p(\theta|X) = \frac{p(X|\theta)p(\theta)}{p(X)} = \frac{1}{C} \cdot f(\theta)$$

ここで：
- $f(\theta) = p(X|\theta)p(\theta)$（計算可能）
- $C = p(X) = \int p(X|\theta)p(\theta)d\theta$（計算困難）

### MH法による見事な解決

MH法の採択確率を見てみましょう：

$$\alpha = \min\left(1, \frac{p(\theta')q(\theta^{(t)}|\theta')}{p(\theta^{(t)})q(\theta'|\theta^{(t)})}\right)$$

比率 $\frac{p(\theta')}{p(\theta^{(t)})}$ を計算すると：

$$\frac{p(\theta')}{p(\theta^{(t)})} = \frac{\frac{1}{C} \cdot f(\theta')}{\frac{1}{C} \cdot f(\theta^{(t)})} = \frac{f(\theta')}{f(\theta^{(t)})}$$

**未知の正規化定数 $C$ が分子と分母で見事に相殺されます！**

### これが意味すること

1. **事後分布の正確な式を知らなくても**サンプリングを進められる
2. **計算困難な積分を回避**できる
3. **比例関係** $p(\theta) \propto f(\theta)$ だけ分かれば十分

これが、MH法が「積分の壁」を回避できる最大の理由です。

## 2.2 基本実装

In [ ]:
def metropolis_hastings(target_log_pdf, proposal_sampler, proposal_log_pdf, 
                       initial_value, n_samples, verbose=False):
    """
    メトロポリス・ヘイスティングス法の汎用実装
    
    Parameters:
    - target_log_pdf: 目標分布の対数確率密度関数
    - proposal_sampler: 提案分布からのサンプラー関数 (current_state) -> proposed_state
    - proposal_log_pdf: 提案分布の対数確率密度関数 (proposed, current) -> log_q
    - initial_value: 初期値
    - n_samples: サンプル数
    - verbose: 詳細情報の表示
    
    Returns:
    - samples: サンプル配列
    - acceptance_rate: 受理率
    - log_probs: 各サンプルの対数確率
    """
    # 初期化
    if np.isscalar(initial_value):
        samples = np.zeros(n_samples)
        dim = 1
    else:
        samples = np.zeros((n_samples, len(initial_value)))
        dim = len(initial_value)
    
    current = np.copy(initial_value)
    current_log_prob = target_log_pdf(current)
    n_accepted = 0
    log_probs = np.zeros(n_samples)
    
    for i in range(n_samples):
        # 新しい状態を提案
        proposed = proposal_sampler(current)
        proposed_log_prob = target_log_pdf(proposed)
        
        # 受理確率を計算（対数スケールで安全に計算）
        log_alpha = (proposed_log_prob + proposal_log_pdf(current, proposed) - 
                    current_log_prob - proposal_log_pdf(proposed, current))
        alpha = min(1.0, np.exp(log_alpha))
        
        # 受理/棄却を決定
        if np.random.rand() < alpha:
            current = proposed
            current_log_prob = proposed_log_prob
            n_accepted += 1
        
        if dim == 1:
            samples[i] = current
        else:
            samples[i] = current
        log_probs[i] = current_log_prob
        
        if verbose and (i + 1) % (n_samples // 10) == 0:
            print(f"Progress: {i+1}/{n_samples}, Acceptance Rate: {n_accepted/(i+1):.3f}")
    
    acceptance_rate = n_accepted / n_samples
    return samples, acceptance_rate, log_probs

## 2.3 例1：混合正規分布からのサンプリング

まず、1次元の混合正規分布からサンプリングしてみましょう。

In [ ]:
# 混合正規分布の定義
def mixture_log_pdf(x):
    """2つの正規分布の混合の対数確率密度"""
    component1 = stats.norm.logpdf(x, -2, 0.5)
    component2 = stats.norm.logpdf(x, 2, 1.0)
    # log(0.3 * exp(component1) + 0.7 * exp(component2))
    max_comp = np.maximum(component1, component2)
    return max_comp + np.log(0.3 * np.exp(component1 - max_comp) + 
                            0.7 * np.exp(component2 - max_comp))

# 対称な提案分布（ランダムウォーク）
def random_walk_sampler(current, step_size=0.5):
    return current + np.random.normal(0, step_size)

def symmetric_proposal_log_pdf(proposed, current):
    return 0.0  # 対称な提案分布の場合、比は1（対数で0）

# サンプリング実行
samples, acceptance_rate, log_probs = metropolis_hastings(
    target_log_pdf=mixture_log_pdf,
    proposal_sampler=lambda x: random_walk_sampler(x, 0.8),
    proposal_log_pdf=symmetric_proposal_log_pdf,
    initial_value=0.0,
    n_samples=10000,
    verbose=True
)

print(f"\n最終受理率: {acceptance_rate:.3f}")

In [ ]:
# 結果の可視化
def plot_mcmc_results_1d(samples, target_log_pdf, burnin=1000, title="MCMC Results"):
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    # トレースプロット
    axes[0, 0].plot(samples[:2000], alpha=0.7, linewidth=0.8)
    axes[0, 0].set_title('Trace Plot (first 2000 samples)')
    axes[0, 0].set_xlabel('Iteration')
    axes[0, 0].set_ylabel('Value')
    axes[0, 0].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
    axes[0, 0].legend()
    
    # ヒストグラムと真の分布の比較
    axes[0, 1].hist(samples[burnin:], bins=60, density=True, alpha=0.7, 
                    color='skyblue', label='MCMC samples')
    x_range = np.linspace(samples.min(), samples.max(), 1000)
    true_density = np.exp(target_log_pdf(x_range))
    axes[0, 1].plot(x_range, true_density, 'r-', linewidth=2, label='True distribution')
    axes[0, 1].set_title('Sample Distribution vs True Distribution')
    axes[0, 1].set_xlabel('Value')
    axes[0, 1].set_ylabel('Density')
    axes[0, 1].legend()
    
    # 自己相関関数
    from statsmodels.tsa.stattools import acf
    lags = min(200, len(samples[burnin:]) // 10)
    autocorr = acf(samples[burnin:], nlags=lags, fft=True)
    axes[0, 2].plot(autocorr)
    axes[0, 2].axhline(0, color='k', linestyle='--', alpha=0.5)
    axes[0, 2].axhline(0.05, color='r', linestyle='--', alpha=0.5, label='5%')
    axes[0, 2].axhline(-0.05, color='r', linestyle='--', alpha=0.5)
    axes[0, 2].set_title('Autocorrelation Function')
    axes[0, 2].set_xlabel('Lag')
    axes[0, 2].set_ylabel('ACF')
    axes[0, 2].legend()
    
    # 累積平均
    cumulative_mean = np.cumsum(samples[burnin:]) / np.arange(1, len(samples[burnin:]) + 1)
    axes[1, 0].plot(cumulative_mean)
    true_mean = np.sum([0.3 * (-2), 0.7 * 2])  # 混合分布の理論平均
    axes[1, 0].axhline(true_mean, color='r', linestyle='--', label=f'True mean = {true_mean:.2f}')
    axes[1, 0].set_title('Cumulative Mean')
    axes[1, 0].set_xlabel('Iteration')
    axes[1, 0].set_ylabel('Mean')
    axes[1, 0].legend()
    
    # 受理率の推移
    window_size = len(samples) // 100
    running_acceptance = []
    for i in range(window_size, len(samples), window_size):
        # 簡易的な受理率計算（連続する値の変化で判定）
        window_samples = samples[i-window_size:i]
        changes = np.sum(np.diff(window_samples) != 0)
        running_acceptance.append(changes / window_size)
    
    axes[1, 1].plot(running_acceptance)
    axes[1, 1].axhline(acceptance_rate, color='r', linestyle='--', 
                       label=f'Overall: {acceptance_rate:.3f}')
    axes[1, 1].set_title('Running Acceptance Rate')
    axes[1, 1].set_xlabel('Window')
    axes[1, 1].set_ylabel('Acceptance Rate')
    axes[1, 1].legend()
    
    # QQプロット（理論分布との比較は困難なので、正規性のテスト）
    from scipy.stats import probplot
    probplot(samples[burnin:], dist="norm", plot=axes[1, 2])
    axes[1, 2].set_title('Q-Q Plot (vs Normal)')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

plot_mcmc_results_1d(samples, mixture_log_pdf, title="Mixture Gaussian Sampling")

## 2.4 提案分布の選択：「歩幅」の科学

提案分布 $q(x'|x^{(t)})$ は、MCMCウォーカーの「歩幅」を決定します。この歩幅の適切なチューニングが、探索効率を劇的に左右する極めて重要な要素です。

### 🚶‍♂️ 歩幅のアナロジー：街歩きからの洞察

imagine you're exploring an unknown city to find popular areas:

**🐌 歩幅が小さすぎる（慎重すぎるウォーカー）:**
- 一歩ずつ慎重に進む → どこに行っても「まあ、いいか」と受け入れる
- ✅ 採択率は高い（ほぼ100%近く）
- ❌ 同じブロックをちまちまと歩き回るだけ
- ❌ 街全体の人気エリアを把握できない（非効率な探索）

**🏃‍♂️ 歩幅が大きすぎる（冒険的すぎるウォーカー）:**
- 大きくジャンプしてみる → でも殆どの場所が「微妙...」で元に戻る
- ❌ 採択率が極端に低い（10%以下）
- ❌ ほぼ同じ場所に留まり続ける
- ❌ 新しい場所を受け入れてもらえない（探索停滞）

**🎯 歩幅が適切（賢いウォーカー）:**
- 程よいステップで着実に探索 → 良い場所は受け入れ、悪い場所は時々スキップ
- ✅ 適度な採択率（理論最適値：1D→44%, 高次元→23%）
- ✅ 街全体を効率よく探索
- ✅ 人気エリアには長く滞在、そうでない場所もバランスよく訪問

### 📊 採択率と探索効率の関係

```
採択率 100% → 移動距離が極小 → 探索が非効率
採択率  50% → 適度な移動距離 → 良好なバランス  
採択率  10% → 移動がほぼ停止 → 探索が停滞
採択率   0% → 完全に停止    → 探索不可能
```

### 🔬 理論最適値の深い意味

最適採択率（1D: 44%, 高次元: 23%）は、**探索距離の2乗平均**（つまり効率）を最大化します：

$$\text{効率} = \lim_{t \to \infty} \frac{E[|X_t - X_0|^2]}{t}$$

この値は、以下の微妙なバランスの最適解：
- **採択頻度**: 新しい場所を受け入れる頻度
- **移動距離**: 一回の採択での移動距離

採択率が高すぎると移動距離が小さく、低すぎると採択頻度が低くなります。

### 🎨 視覚的理解：歩幅と軌跡パターン

異なる歩幅設定での典型的なトレースプロット：

```
小さい歩幅: ~~~~~~~~ (細かい振動、局所的)
適切な歩幅: ∩∪∩∪∩∪ (適度な起伏、広域探索)  
大きい歩幅: ------   (平坦、停滞)
```

次のセクションで、これらの違いを実際のデータで確認してみましょう。

In [ ]:
def compare_step_sizes(step_sizes, n_samples=5000):
    """
    異なるステップサイズでの性能比較
    """
    results = {}
    
    for step_size in step_sizes:
        print(f"Testing step size: {step_size}")
        
        samples, acc_rate, _ = metropolis_hastings(
            target_log_pdf=mixture_log_pdf,
            proposal_sampler=lambda x: random_walk_sampler(x, step_size),
            proposal_log_pdf=symmetric_proposal_log_pdf,
            initial_value=0.0,
            n_samples=n_samples
        )
        
        # 有効サンプルサイズの計算（自己相関を考慮）
        from statsmodels.tsa.stattools import acf
        burnin = n_samples // 5
        autocorr = acf(samples[burnin:], nlags=min(200, len(samples[burnin:])//4), fft=True)
        
        # 最初に0.05を下回るラグを見つける
        tau_int = 1
        for lag in range(1, len(autocorr)):
            if autocorr[lag] < 0.05:
                tau_int = lag
                break
        
        eff_sample_size = len(samples[burnin:]) / (2 * tau_int + 1)
        
        results[step_size] = {
            'samples': samples,
            'acceptance_rate': acc_rate,
            'autocorr_time': tau_int,
            'eff_sample_size': eff_sample_size,
            'efficiency': eff_sample_size * acc_rate  # 総合効率の指標
        }
    
    return results

# 異なるステップサイズで比較
step_sizes = [0.1, 0.5, 1.0, 2.0, 5.0]
comparison_results = compare_step_sizes(step_sizes)

# 結果の表示
print("\n=== Step Size Comparison ===")
print(f"{'Step Size':<10} {'Acc Rate':<10} {'Autocorr':<10} {'Eff Size':<12} {'Efficiency':<12}")
print("-" * 60)
for step_size in step_sizes:
    result = comparison_results[step_size]
    print(f"{step_size:<10.1f} {result['acceptance_rate']:<10.3f} "
          f"{result['autocorr_time']:<10d} {result['eff_sample_size']:<12.1f} "
          f"{result['efficiency']:<12.1f}")

In [ ]:
# ステップサイズ比較の包括的可視化
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

# パフォーマンス指標の抽出
step_sizes_plot = list(comparison_results.keys())
acc_rates = [comparison_results[s]['acceptance_rate'] for s in step_sizes_plot]
autocorr_times = [comparison_results[s]['autocorr_time'] for s in step_sizes_plot]
eff_sizes = [comparison_results[s]['eff_sample_size'] for s in step_sizes_plot]
efficiencies = [comparison_results[s]['efficiency'] for s in step_sizes_plot]

# 1. 受理率 vs ステップサイズ
axes[0, 0].semilogx(step_sizes_plot, acc_rates, 'bo-', markersize=8, linewidth=2)
axes[0, 0].axhline(0.44, color='red', linestyle='--', alpha=0.8, linewidth=2, label='理論最適 (44%)')
axes[0, 0].axhspan(0.2, 0.7, alpha=0.2, color='green', label='推奨範囲')
axes[0, 0].set_xlabel('ステップサイズ', fontsize=12)
axes[0, 0].set_ylabel('受理率', fontsize=12)
axes[0, 0].set_title('🎯 受理率 vs ステップサイズ', fontsize=14)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 自己相関時間 vs ステップサイズ
axes[0, 1].semilogx(step_sizes_plot, autocorr_times, 'go-', markersize=8, linewidth=2)
axes[0, 1].set_xlabel('ステップサイズ', fontsize=12)
axes[0, 1].set_ylabel('自己相関時間', fontsize=12)
axes[0, 1].set_title('⏱️ 自己相関時間 vs ステップサイズ', fontsize=14)
axes[0, 1].grid(True, alpha=0.3)

# 3. 有効サンプルサイズ vs ステップサイズ
axes[0, 2].semilogx(step_sizes_plot, eff_sizes, 'mo-', markersize=8, linewidth=2)
axes[0, 2].set_xlabel('ステップサイズ', fontsize=12)
axes[0, 2].set_ylabel('有効サンプルサイズ', fontsize=12)
axes[0, 2].set_title('📊 有効サンプルサイズ vs ステップサイズ', fontsize=14)
axes[0, 2].grid(True, alpha=0.3)

# 4. 総合効率 vs ステップサイズ
axes[0, 3].semilogx(step_sizes_plot, efficiencies, 'ro-', markersize=8, linewidth=2)
optimal_idx = np.argmax(efficiencies)
axes[0, 3].scatter(step_sizes_plot[optimal_idx], efficiencies[optimal_idx], 
                  color='gold', s=200, marker='*', zorder=5, label=f'最適値 ({step_sizes_plot[optimal_idx]})')
axes[0, 3].set_xlabel('ステップサイズ', fontsize=12)
axes[0, 3].set_ylabel('総合効率', fontsize=12)
axes[0, 3].set_title('⭐ 総合効率 vs ステップサイズ', fontsize=14)
axes[0, 3].legend()
axes[0, 3].grid(True, alpha=0.3)

# 5-8. トレースプロットの比較（最初の1000サンプル）
colors = ['blue', 'green', 'orange', 'red']
step_examples = [0.1, 0.5, 1.0, 5.0]
titles = ['🐌 過小ステップ (0.1)', '✅ 適切ステップ (0.5)', '⚡ やや大ステップ (1.0)', '🏃‍♂️ 過大ステップ (5.0)']

for i, (step_size, title, color) in enumerate(zip(step_examples, titles, colors)):
    if step_size in comparison_results:
        samples = comparison_results[step_size]['samples'][:1000]
        acc_rate = comparison_results[step_size]['acceptance_rate']
        autocorr = comparison_results[step_size]['autocorr_time']
        
        axes[1, i].plot(samples, alpha=0.8, color=color, linewidth=1)
        axes[1, i].set_title(f'{title}\nAcc:{acc_rate:.3f}, ACT:{autocorr}', fontsize=11)
        axes[1, i].set_xlabel('イテレーション')
        axes[1, i].set_ylabel('値')
        axes[1, i].grid(True, alpha=0.3)

# 9-12. ヒストグラムの比較
x_range = np.linspace(-6, 6, 1000)
true_density = np.exp(mixture_log_pdf(x_range))

for i, (step_size, color) in enumerate(zip(step_examples, colors)):
    if step_size in comparison_results:
        samples = comparison_results[step_size]['samples'][1000:]  # burnin除去
        
        axes[2, i].hist(samples, bins=50, density=True, alpha=0.7, color=color, 
                       edgecolor='black', linewidth=0.5)
        axes[2, i].plot(x_range, true_density, 'r-', linewidth=3, label='真の分布')
        axes[2, i].set_title(f'ステップ {step_size}: 分布比較', fontsize=11)
        axes[2, i].set_xlabel('値')
        axes[2, i].set_ylabel('密度')
        axes[2, i].legend()

plt.tight_layout()
plt.suptitle('🔬 ステップサイズ最適化の包括的分析', fontsize=16, y=1.02)
plt.show()

# パフォーマンステーブルの表示
print("\n" + "="*80)
print("📈 ステップサイズ最適化レポート")
print("="*80)
print(f"{'ステップ':<8} {'受理率':<8} {'自己相関':<10} {'有効サンプル':<12} {'総合効率':<10} {'評価':<15}")
print("-"*80)

for i, step_size in enumerate(step_sizes_plot):
    result = comparison_results[step_size]
    acc_rate = result['acceptance_rate']
    
    # 評価ロジック
    if 0.35 <= acc_rate <= 0.55:
        rating = "✅ 優秀"
    elif 0.25 <= acc_rate <= 0.65:
        rating = "⚡ 良好"
    elif 0.15 <= acc_rate <= 0.75:
        rating = "⚠️ 注意"
    else:
        rating = "❌ 不良"
    
    print(f"{step_size:<8.1f} {acc_rate:<8.3f} {result['autocorr_time']:<10d} "
          f"{result['eff_sample_size']:<12.1f} {result['efficiency']:<10.1f} {rating:<15}")

print("-"*80)
print("💡 最適化のガイドライン:")
print("• 受理率 35-55%: 理想的な範囲")
print("• 受理率 25-65%: 許容範囲") 
print("• 受理率 < 25% or > 65%: 要調整")
print("• 自己相関時間: 小さいほど良い")
print("• 総合効率: 受理率 × 有効サンプルサイズ")

## 2.5 例2：非対称な提案分布

今度は非対称な提案分布を使った例を見てみましょう。

In [ ]:
# 指数分布からのサンプリング
def exponential_log_pdf(x, rate=1.0):
    """指数分布の対数確率密度"""
    if x < 0:
        return -np.inf
    return np.log(rate) - rate * x

# 非対称な提案分布（対数正規分布）
def lognormal_proposal_sampler(current, sigma=0.5):
    """対数正規分布による提案"""
    return current * np.exp(np.random.normal(0, sigma))

def lognormal_proposal_log_pdf(proposed, current, sigma=0.5):
    """対数正規提案分布の対数確率密度"""
    if proposed <= 0 or current <= 0:
        return -np.inf
    log_ratio = np.log(proposed / current)
    return -0.5 * (log_ratio / sigma)**2 - 0.5 * np.log(2 * np.pi * sigma**2) - np.log(proposed)

# サンプリング実行
rate_param = 2.0
samples_exp, acceptance_rate_exp, _ = metropolis_hastings(
    target_log_pdf=lambda x: exponential_log_pdf(x, rate_param),
    proposal_sampler=lambda x: lognormal_proposal_sampler(x, 0.3),
    proposal_log_pdf=lambda p, c: lognormal_proposal_log_pdf(p, c, 0.3),
    initial_value=1.0,
    n_samples=10000,
    verbose=True
)

print(f"\n指数分布サンプリング受理率: {acceptance_rate_exp:.3f}")
print(f"理論平均: {1/rate_param:.3f}, サンプル平均: {np.mean(samples_exp[2000:]):.3f}")

In [ ]:
# 指数分布サンプリング結果の可視化
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

burnin = 2000

# トレースプロット
axes[0, 0].plot(samples_exp[:3000], alpha=0.7)
axes[0, 0].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
axes[0, 0].set_title('Trace Plot')
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('Value')
axes[0, 0].legend()

# ヒストグラムと真の分布
axes[0, 1].hist(samples_exp[burnin:], bins=50, density=True, alpha=0.7, 
                color='lightblue', label='MCMC samples')
x_range = np.linspace(0, np.percentile(samples_exp[burnin:], 95), 1000)
true_density = rate_param * np.exp(-rate_param * x_range)
axes[0, 1].plot(x_range, true_density, 'r-', linewidth=2, label='True exponential')
axes[0, 1].set_title('Sample Distribution vs True Distribution')
axes[0, 1].set_xlabel('Value')
axes[0, 1].set_ylabel('Density')
axes[0, 1].legend()

# Q-Qプロット（指数分布と比較）
from scipy.stats import probplot
probplot(samples_exp[burnin:], dist=stats.expon, sparams=(0, 1/rate_param), plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot vs Exponential Distribution')

# 累積分布関数の比較
sorted_samples = np.sort(samples_exp[burnin:])
empirical_cdf = np.arange(1, len(sorted_samples) + 1) / len(sorted_samples)
theoretical_cdf = 1 - np.exp(-rate_param * sorted_samples)

axes[1, 1].plot(sorted_samples, empirical_cdf, 'b-', alpha=0.7, label='Empirical CDF')
axes[1, 1].plot(sorted_samples, theoretical_cdf, 'r-', linewidth=2, label='Theoretical CDF')
axes[1, 1].set_title('CDF Comparison')
axes[1, 1].set_xlabel('Value')
axes[1, 1].set_ylabel('Cumulative Probability')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 2.6 多変量分布への拡張

2次元の多変量正規分布からサンプリングしてみましょう。

In [ ]:
# 2次元多変量正規分布
def multivariate_normal_log_pdf(x, mu, cov):
    """多変量正規分布の対数確率密度"""
    k = len(mu)
    diff = x - mu
    
    # 数値安定性のための計算
    try:
        chol = np.linalg.cholesky(cov)
        log_det = 2 * np.sum(np.log(np.diag(chol)))
        solve = np.linalg.solve(chol, diff)
        mahalanobis_sq = np.sum(solve**2)
    except np.linalg.LinAlgError:
        return -np.inf
    
    return -0.5 * (k * np.log(2 * np.pi) + log_det + mahalanobis_sq)

# 多変量提案分布
def multivariate_proposal_sampler(current, cov_proposal):
    """多変量正規提案分布"""
    return np.random.multivariate_normal(current, cov_proposal)

def multivariate_proposal_log_pdf(proposed, current, cov_proposal):
    """多変量正規提案分布の対数確率密度（対称なので0）"""
    return 0.0

# パラメータ設定
mu_target = np.array([1.0, 2.0])
cov_target = np.array([[1.0, 0.7], [0.7, 2.0]])
cov_proposal = 0.5 * np.eye(2)

# サンプリング実行
samples_mv, acceptance_rate_mv, _ = metropolis_hastings(
    target_log_pdf=lambda x: multivariate_normal_log_pdf(x, mu_target, cov_target),
    proposal_sampler=lambda x: multivariate_proposal_sampler(x, cov_proposal),
    proposal_log_pdf=lambda p, c: multivariate_proposal_log_pdf(p, c, cov_proposal),
    initial_value=np.array([0.0, 0.0]),
    n_samples=10000,
    verbose=True
)

print(f"\n多変量正規分布サンプリング受理率: {acceptance_rate_mv:.3f}")
print(f"理論平均: {mu_target}")
print(f"サンプル平均: {np.mean(samples_mv[2000:], axis=0)}")
print(f"理論共分散:\n{cov_target}")
print(f"サンプル共分散:\n{np.cov(samples_mv[2000:].T)}")

In [ ]:
# 多変量結果の可視化
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

burnin = 2000
samples_clean = samples_mv[burnin:]

# 散布図
axes[0, 0].scatter(samples_clean[:, 0], samples_clean[:, 1], alpha=0.6, s=1)
axes[0, 0].set_xlabel('X1')
axes[0, 0].set_ylabel('X2')
axes[0, 0].set_title('Scatter Plot of Samples')
axes[0, 0].set_aspect('equal')

# 等高線プロット
x1_range = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 50)
x2_range = np.linspace(samples_clean[:, 1].min(), samples_clean[:, 1].max(), 50)
X1, X2 = np.meshgrid(x1_range, x2_range)
pos = np.dstack((X1, X2))

# 真の分布の等高線
rv = stats.multivariate_normal(mu_target, cov_target)
axes[0, 1].contour(X1, X2, rv.pdf(pos), colors='red', alpha=0.8)
axes[0, 1].scatter(samples_clean[::10, 0], samples_clean[::10, 1], alpha=0.3, s=1)
axes[0, 1].set_xlabel('X1')
axes[0, 1].set_ylabel('X2')
axes[0, 1].set_title('Samples with True Distribution Contours')

# マージナル分布
axes[0, 2].hist(samples_clean[:, 0], bins=50, density=True, alpha=0.7, label='X1 samples')
x1_theory = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 100)
axes[0, 2].plot(x1_theory, stats.norm.pdf(x1_theory, mu_target[0], np.sqrt(cov_target[0, 0])), 
                'r-', label='X1 true')
axes[0, 2].set_title('Marginal Distribution X1')
axes[0, 2].legend()

# トレースプロット
axes[1, 0].plot(samples_mv[:3000, 0], alpha=0.7, label='X1')
axes[1, 0].plot(samples_mv[:3000, 1], alpha=0.7, label='X2')
axes[1, 0].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
axes[1, 0].set_title('Trace Plot')
axes[1, 0].set_xlabel('Iteration')
axes[1, 0].set_ylabel('Value')
axes[1, 0].legend()

# 第2マージナル分布
axes[1, 1].hist(samples_clean[:, 1], bins=50, density=True, alpha=0.7, label='X2 samples')
x2_theory = np.linspace(samples_clean[:, 1].min(), samples_clean[:, 1].max(), 100)
axes[1, 1].plot(x2_theory, stats.norm.pdf(x2_theory, mu_target[1], np.sqrt(cov_target[1, 1])), 
                'r-', label='X2 true')
axes[1, 1].set_title('Marginal Distribution X2')
axes[1, 1].legend()

# 相関の収束
n_points = len(samples_clean)
window_size = n_points // 100
correlations = []
for i in range(window_size, n_points, window_size):
    window_samples = samples_clean[i-window_size:i]
    corr = np.corrcoef(window_samples[:, 0], window_samples[:, 1])[0, 1]
    correlations.append(corr)

axes[1, 2].plot(correlations)
true_corr = cov_target[0, 1] / np.sqrt(cov_target[0, 0] * cov_target[1, 1])
axes[1, 2].axhline(true_corr, color='red', linestyle='--', 
                   label=f'True correlation = {true_corr:.3f}')
axes[1, 2].set_title('Running Correlation')
axes[1, 2].set_xlabel('Window')
axes[1, 2].set_ylabel('Correlation')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

## 2.7 演習問題

### 問題1：ベータ分布からのサンプリング
ベータ分布 $\text{Beta}(\alpha=2, \beta=5)$ からメトロポリス・ヘイスティングス法でサンプリングしなさい。

In [ ]:
# 問題1の解答欄
def beta_log_pdf(x, alpha=2, beta=5):
    """ベータ分布の対数確率密度"""
    if x <= 0 or x >= 1:
        return -np.inf
    return (alpha - 1) * np.log(x) + (beta - 1) * np.log(1 - x)

# ここに実装してください
# ヒント：[0,1]区間に制約があるので、提案が範囲外の場合の処理が必要

pass  # 学習者が実装

### 問題2：最適な受理率の調査
1次元正規分布に対して、異なるステップサイズで受理率と効率を調べ、最適な受理率（約23%）が実際に効率的かを確認しなさい。

In [ ]:
# 問題2の解答欄
def standard_normal_log_pdf(x):
    return -0.5 * x**2 - 0.5 * np.log(2 * np.pi)

# ここに実装してください
# ヒント：複数のステップサイズで実験し、受理率と自己相関時間の関係を調べる

pass  # 学習者が実装

## まとめ：MH法マスターへの道

この章では、メトロポリス・ヘイスティングス法について以下を包括的に学習しました：

### 🧠 核心的理解

1. **革命的アイデア**：
   - 正規化定数の巧妙な相殺メカニズム
   - 詳細釣り合い条件による理論的保証
   - 「賢いランダムウォーク」としての直感的理解

2. **数学的美しさ**：
   - 受理確率α の導出と物理学的解釈
   - 確率比による効率的計算
   - 対数スケールでの数値安定性

### 🔧 実践的スキル

3. **提案分布の設計哲学**：
   - **ステップサイズ**: MCMCウォーカーの「歩幅」
   - **最適受理率**: 1D→44%, 高次元→23%の理論的根拠
   - **効率指標**: 自己相関時間と有効サンプルサイズ

4. **性能診断の体系**：
   - トレースプロット：混合と収束の視覚的確認
   - 受理率モニタリング：リアルタイム性能評価
   - 分布比較：理論値との整合性検証

### 🚀 実装における重要ポイント

**技術的ベストプラクティス:**
- **対数スケール計算**: 数値アンダーフロー回避
- **対称性の活用**: 計算コスト削減
- **境界条件の処理**: 制約付き分布での実装
- **適応的調整**: 受理率に基づく自動チューニング

**設計哲学:**
- **汎用性**: あらゆる連続分布に適用可能
- **堅牢性**: パラメータ誤設定に対する許容度
- **拡張性**: 高次元問題への自然な拡張

### 🎯 応用戦略

**分布特性に応じた戦略:**
- **単峰性分布**: 標準的ランダムウォーク
- **多峰性分布**: 大きめステップサイズ
- **制約付き分布**: 境界反射または変数変換
- **高次元分布**: 成分別更新またはブロック更新

### 🔮 次のステップ

MH法をマスターしたあなたは、MCMCの**基盤技術**を完全に理解しました。

**Chapter 3（ギブス サンプリング）**では：
- MH法の**特殊化による効率化**
- 条件付き分布の活用戦略
- 高次元問題での実用的解決策
- 階層ベイズモデルでの威力

**Chapter 4（収束診断）**では：
- MH法の**品質保証システム**
- 自動診断による信頼性向上
- 実際の研究での活用法

### 💡 重要な哲学

> MH法は「完璧な解」を求めるのではなく、「十分に良い近似」を効率的に得る手法です。この哲学は、現代のデータサイエンス全体に通じる重要な考え方です。

**記憶すべき金言:**
- 「受理率44%は拒絶を恐れない勇気の証」
- 「完璧な提案より、適切な提案を継続する」
- 「収束は目標ではなく、探索の質を示す指標」

あなたは今、MCMCの心臓部であるMH法を使いこなせるようになりました。次はより洗練された手法で、さらに高い効率を追求していきましょう！